In [2]:
from dotenv import load_dotenv
import os
from qiskit import *
from qiskit_ibm_provider import IBMProvider
from math import *
import qiskit

load_dotenv()
IBM_KEY = os.getenv("API_KEY")
provider = IBMProvider()

## Quantum Teleportation



In [4]:
q = QuantumRegister(3)
ca = ClassicalRegister(2)
cb = ClassicalRegister(1)

qc = QuantumCircuit(q, ca, cb)
# Creating entaglement
qc.h(q[1])
qc.cx(q[1], q[2])

# Creating a dummy state that is to be teleported
qc.h(q[0])
qc.cx(q[0], q[1])
qc.h(q[0])
qc.measure(q[0], ca[0])
qc.measure(q[1], ca[1])
qc.barrier()

# Controlled on the o/p receieved by Bob, he performs X gate and Z gate
qc.x(q[2]).c_if(ca[1] ,1)
qc.z(q[2]).c_if(ca[0] ,1)

# Check if state obtained is same as one to be teleported (H^{dag} = H), So output on measuring q[2] should always be |0>
qc.h(q[2])
qc.measure(q[2], cb[0])

qc.draw()

┌───┐          ┌───┐┌─┐ ░                                 
q1_0: ┤ H ├───────■──┤ H ├┤M├─░─────────────────────────────────
      ├───┤     ┌─┴─┐└┬─┬┘└╥┘ ░                                 
q1_1: ┤ H ├──■──┤ X ├─┤M├──╫──░─────────────────────────────────
      └───┘┌─┴─┐└───┘ └╥┘  ║  ░    ┌───┐       ┌───┐    ┌───┐┌─┐
q1_2: ─────┤ X ├───────╫───╫──░────┤ X ├───────┤ Z ├────┤ H ├┤M├
           └───┘       ║   ║  ░    └─╥─┘       └─╥─┘    └───┘└╥┘
                       ║   ║    ┌────╨─────┐┌────╨─────┐      ║ 
c2: 2/═════════════════╩═══╩════╡ c2_1=0x1 ╞╡ c2_0=0x1 ╞══════╬═
                       1   0    └──────────┘└──────────┘      ║ 
c3: 1/════════════════════════════════════════════════════════╩═
                                                              0

In [5]:
backend = Aer.get_backend('qasm_simulator')
qjob = execute(qc, backend)
counts = qjob.result().get_counts()
print(counts)
# Output structure is cb[0] ca[0]ca[1]
# Always get 0 for q[2] 

{'0 11': 250, '0 00': 254, '0 01': 253, '0 10': 267}
